# Mikio's Zelus Assessment

## Question 0
#### We don’t expect you to have any cricket knowledge and that is not a requirement to ace this assessment. But we understand that familiarity with cricket may vary from one candidate to the next so we would like to know how you would rate your knowledge of cricket from 1 to 5, where 1 is basically no knowledge (like you had never seen or read anything about the sport until the days before this assessment) and 5 is highly knowledgeable (you watch matches regularly and have a jersey for the Rajasthan Royals in your closet, for example)

**My Response: 1
While I am aware of cricket's popularity in countries like Britain and India, I have never actively engaged with the sport through reading about it or watching matches. Therefore, I would rate my knowledge of cricket as a 1 for this question.**

## Question 1
#### Develop a batch data ingest process to load the ODI match results and ball-by-ball innings data to a database of your choosing (as a default, you can use SQLite). The solution should include a step that downloads files directly from cricsheet.org and performs any required preprocessing. The database schema should store match results and ball-by-ball innings data along with the universe of players that appear across all matches. The process should be runnable from the command line, inclusive of creating any dependencies (e.g. local file directories, the database, etc.). Please include a README.md file with instructions on how to build and run the ingest process to reproduce your results.

In [1]:
import requests
import zipfile
import os
import json
import pandas as pd
from sqlalchemy import create_engine
import hashlib
import time
import sqlite3
from sqlalchemy import create_engine

The `download_and_extract_data` function downloads a ZIP file from a given URL and extracts its contents.
    
Args:
- url (str): URL of the ZIP file to download.
- zip_path (str): Path where the downloaded ZIP file will be saved.
- extract_dir (str): Directory where the contents of the ZIP file will be extracted.

In [2]:
def download_and_extract_data(url, zip_path, extract_dir):
    # Send a GET request to the specified URL to download the ZIP file
    response = requests.get(url)
    
    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Open the specified ZIP file path in write-binary mode
        with open(zip_path, 'wb') as file:
            # Write the content of the response (ZIP file) to the file
            file.write(response.content)
        print("Downloaded ZIP file.")
        
        # Extract the contents of the ZIP file
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            # Extract all files into the specified directory
            zip_ref.extractall(extract_dir)
        print(f"Extracted contents to {extract_dir}")
        
        # Remove the downloaded ZIP file after extraction
        os.remove(zip_path)
        print("Removed ZIP file.")
    else:
        # If the request was not successful, print an error message
        print(f"Failed to download data: {response.status_code}")

In [3]:
url = "https://cricsheet.org/downloads/odis_json.zip"
zip_path = "odis_json_data.zip"
extract_dir = "odis_json_data"
download_and_extract_data(url, zip_path, extract_dir)

Downloaded ZIP file.
Extracted contents to odis_json_data
Removed ZIP file.


######  Match Data

The `extract_match_info` function is designed to parse data from a JSON file and extract key match information. This information is essential for answering specific questions in the assessment, particularly regarding win records and win percentages for different teams by year and gender. The function extracts the following details to answer the corresponding questions:

1. The win records (percentage win and total wins) for each team by year and gender, excluding ties, matches with no result, and matches decided by the DLS method.
- `Year`: Extracted from the date field to group the data by year.
- `Gender`: Needed to differentiate between male and female teams.
- `Teams`: To identify which teams played the match.
- `Winner`: To determine the winning team for each match.
- `Result`: To exclude matches with no result or ties.
- `Outcome by method`: To exclude matches decided by the DLS method, if this information is available in the data.
2. Which male and female teams had the highest win percentages in 2019:
- `Year`: To filter the data for matches played in 2019.
- `Gender`: To differentiate between male and female teams.
- `Teams`: To identify which teams played the match.
- `Winner`: To determine the winning team for each match.

In [4]:
def extract_match_info(data):
    match_info = {
        'match_id': data['info']['match_type_number'],
        'event_name': data['info'].get('event', {}).get('name', None),
        'date': data['info']['dates'][0],
        'gender': data['info']['gender'],
        'team1': data['info']['teams'][0],
        'team2': data['info']['teams'][1],
        'winner': data['info']['outcome'].get('winner', None),
        'result': data['info']['outcome'].get('result', None),
        'win_by_method': data['info']['outcome'].get('by', {}).get('method', None)
    }
    return pd.DataFrame([match_info])

In [5]:
# Initialize an empty DataFrame to store all match data
all_matches_df = pd.DataFrame()

# Iterate over each JSON file in the odis_json_data directory
for filename in os.listdir('odis_json_data'):
    if filename.endswith('.json'):
        filepath = os.path.join('odis_json_data', filename)
        with open(filepath) as file:
            data = json.load(file)
        # Extract match info and append it to the all_matches_df DataFrame
        match_df = extract_match_info(data)
        all_matches_df = pd.concat([all_matches_df, match_df], ignore_index=True)

In [6]:
# Display the combined DataFrame
all_matches_df.head()

,match_id,event_name,date,gender,team1,team2,winner,result,win_by_method
0,4714,India tour of South Africa,2023-12-19,male,India,South Africa,South Africa,None,None
1,3070,New Zealand in India ODI Series,2010-11-28,male,India,New Zealand,India,None,None
2,2811,Chappell-Hadlee Trophy,2009-02-01,male,Australia,New Zealand,New Zealand,None,None
3,1191,South Africa Women tour of India,2021-03-09,female,South Africa,India,India,None,None
4,4406,ICC Men's Cricket World Cup League 2,2022-06-11,male,Nepal,United States of America,None,tie,None


In [7]:
# Get unique values for the 'result' column
unique_results = all_matches_df['result'].unique()
print("Unique results:", unique_results)

# Get unique values for the 'win_by_method' column
unique_win_by_methods = all_matches_df['win_by_method'].unique()
print("Unique win_by_methods:", unique_win_by_methods)

Unique results: [None 'tie' 'no result']
Unique win_by_methods: [None]


###### Analysis of Unique result and win_by_method Values

Based on the output from the dataset, the `result` column has three unique values:
- None: Indicates that there was a clear winner in the match.
- 'tie': Indicates that the match ended in a tie.
- 'no result': Indicates that the match was abandoned or could not be completed.

The `win_by_method` column has only one unique value:
- None: Suggests that the dataset does not contain information about the method of winning (e.g., by runs, by wickets, by DLS method, etc.) for any of the matches. As such, I will drop this column.

###### Implications for Analysis:
For the analysis of win records excluding ties, matches with no result, and matches decided by the DLS method:
- Matches where result is 'tie' or 'no result' should be filtered out.
- It appears that we cannot specifically exclude matches decided by the DLS method based on this dataset, as the `win_by_method` column does not provide this information.

In [8]:
# Drop the win_by_method column as it contains only None values
all_matches_df.drop(columns=['win_by_method'], inplace=True)
# I modified the function as well accordingly
all_matches_df.head()

,match_id,event_name,date,gender,team1,team2,winner,result
0,4714,India tour of South Africa,2023-12-19,male,India,South Africa,South Africa,None
1,3070,New Zealand in India ODI Series,2010-11-28,male,India,New Zealand,India,None
2,2811,Chappell-Hadlee Trophy,2009-02-01,male,Australia,New Zealand,New Zealand,None
3,1191,South Africa Women tour of India,2021-03-09,female,South Africa,India,India,None
4,4406,ICC Men's Cricket World Cup League 2,2022-06-11,male,Nepal,United States of America,None,tie


In [9]:
# Check for duplicate match_id values
duplicate_matches = all_matches_df[all_matches_df.duplicated(subset='match_id', keep=False)]

# Display the duplicate matches, if any
if not duplicate_matches.empty:
    print("Duplicate match_id values found:")
    print(duplicate_matches)
else:
    print("No duplicate match_id values found.")

# Get the count of duplicated matches
duplicate_count = len(duplicate_matches)

# Display the count of duplicate matches
print(f"Count of duplicated matches: {duplicate_count}")

Duplicate match_id values found:
      match_id                         event_name        date  gender  \
0         4714         India tour of South Africa  2023-12-19    male   
741       3780                               None  2016-09-25    male   
892       1188  England Women tour of New Zealand  2021-02-26  female   
1148      3780    New Zealand in India ODI Series  2016-10-29    male   
1260      1188  England Women tour of New Zealand  2021-02-23  female   
1457      4714     Bangladesh tour of New Zealand  2023-12-20    male   

             team1         team2        winner result  
0            India  South Africa  South Africa   None  
741   South Africa       Ireland  South Africa   None  
892    New Zealand       England       England   None  
1148         India   New Zealand         India   None  
1260   New Zealand       England       England   None  
1457    Bangladesh   New Zealand   New Zealand   None  
Count of duplicated matches: 6


I initially utilized the `match_type_number` from the JSON data as the `match_id`. However, upon detecting 6 duplicate matches, it became evident that `match_type_number` alone does not serve as a reliable unique identifier.

To address this issue, I modified the `match_id` to include additional information, such as the date and team names. This aimed to create a more granular and unique identifier for each match.

In [10]:
def extract_match_info(data):
    # Create a unique match_id using the date and team names
    match_id = f"{data['info']['dates'][0]}_{data['info']['teams'][0]}_vs_{data['info']['teams'][1]}"

    match_info = {
        'match_id': match_id,
        'date': data['info']['dates'][0],
        'gender': data['info']['gender'],
        'team1': data['info']['teams'][0],
        'team2': data['info']['teams'][1],
        'winner': data['info']['outcome'].get('winner', None),
        'result': data['info']['outcome'].get('result', None)
    }
    return pd.DataFrame([match_info])

# Initialize an empty DataFrame to store all match data
all_matches_df = pd.DataFrame()

# Iterate over each JSON file in the odis_json_data directory
for filename in os.listdir('odis_json_data'):
    if filename.endswith('.json'):
        filepath = os.path.join('odis_json_data', filename)
        with open(filepath) as file:
            data = json.load(file)
        # Extract match info and append it to the all_matches_df DataFrame
        match_df = extract_match_info(data)
        all_matches_df = pd.concat([all_matches_df, match_df], ignore_index=True)

# Check for duplicate match_id values
duplicate_matches = all_matches_df[all_matches_df.duplicated(subset='match_id', keep=False)]

# Display the duplicate matches, if any
if not duplicate_matches.empty:
    print("Duplicate match_id values found:")
    print(duplicate_matches)
else:
    print("No duplicate match_id values found.")

# Get the count of duplicated matches
duplicate_count = len(duplicate_matches)

# Display the count of duplicate matches
print(f"Count of duplicated matches: {duplicate_count}")

Duplicate match_id values found:
                              match_id        date  gender      team1  \
329    2014-01-19_Australia_vs_England  2014-01-19  female  Australia   
482    2014-01-26_Australia_vs_England  2014-01-26  female  Australia   
674   2018-02-10_India_vs_South Africa  2018-02-10    male      India   
1372   2014-01-26_Australia_vs_England  2014-01-26    male  Australia   
1717  2018-02-07_India_vs_South Africa  2018-02-07  female      India   
1875  2018-02-10_India_vs_South Africa  2018-02-10  female      India   
2069   2014-01-19_Australia_vs_England  2014-01-19    male  Australia   
2317  2018-02-07_India_vs_South Africa  2018-02-07    male      India   

             team2        winner result  
329        England       England   None  
482        England     Australia   None  
674   South Africa  South Africa   None  
1372       England     Australia   None  
1717  South Africa         India   None  
1875  South Africa  South Africa   None  
2069       Engl

In [11]:
selected_matches = all_matches_df[all_matches_df['match_id'] == '2014-01-19_Australia_vs_England']
selected_matches

,match_id,date,gender,team1,team2,winner,result
329,2014-01-19_Australia_vs_England,2014-01-19,female,Australia,England,England,None
2069,2014-01-19_Australia_vs_England,2014-01-19,male,Australia,England,Australia,None


After modifying the `match_id`, the count of duplicated matches increased to 8. This unexpected increase highlighted that the original method of identifying matches was insufficiently granular, leading to some matches being incorrectly considered unique.

Upon further investigation, I noticed that some duplicates differed only by gender. Therefore, I included `gender` in the `match_id`, which effectively resolved the issue, eliminating all duplicate matches.

In [12]:
# Adding gender as part of match_id
def extract_match_info(data):
    match_info = {
        'match_id': f"{data['info']['dates'][0]}_{data['info']['teams'][0]}_vs_{data['info']['teams'][1]}_{data['info']['gender']}",
        'date': data['info']['dates'][0],
        'gender': data['info']['gender'],
        'team1': data['info']['teams'][0],
        'team2': data['info']['teams'][1],
        'winner': data['info']['outcome'].get('winner', None),
        'result': data['info']['outcome'].get('result', None)
    }
    return pd.DataFrame([match_info])

# Initialize an empty DataFrame to store all match data
all_matches_df = pd.DataFrame()

# Iterate over each JSON file in the odis_json_data directory
for filename in os.listdir('odis_json_data'):
    if filename.endswith('.json'):
        filepath = os.path.join('odis_json_data', filename)
        with open(filepath) as file:
            data = json.load(file)
        # Extract match info and append it to the all_matches_df DataFrame
        match_df = extract_match_info(data)
        all_matches_df = pd.concat([all_matches_df, match_df], ignore_index=True)

# Check for duplicate match_id values
duplicate_matches = all_matches_df[all_matches_df.duplicated(subset='match_id', keep=False)]

# Display the duplicate matches, if any
if not duplicate_matches.empty:
    print("Duplicate match_id values found:")
    print(duplicate_matches)
else:
    print("No duplicate match_id values found.")

# Get the count of duplicated matches
duplicate_count = len(duplicate_matches)

# Display the count of duplicate matches
print(f"Count of duplicated matches: {duplicate_count}")

No duplicate match_id values found.
Count of duplicated matches: 0


###### Creating a Numerical Match ID:

To create a unique and efficient identifier for each cricket match, I implemented a hashing function that generates a numerical ID based on the match's date, teams, and gender. This approach ensures consistency, as the same input always produces the same ID, and enhances database efficiency by using integers instead of strings. The numerical ID simplifies data analysis and ensures uniqueness, reducing the risk of different matches having the same identifier.

In [13]:
def create_numerical_id(match_info):
    match_str = f"{match_info['date']}_{match_info['team1']}_{match_info['team2']}_{match_info['gender']}"
    return int(hashlib.sha1(match_str.encode()).hexdigest(), 16) % (10 ** 8)

In [14]:
# Use the create_numerical_id function to generate match_id
def extract_match_info(data):
    match_info = {
        'date': data['info']['dates'][0],
        'gender': data['info']['gender'],
        'team1': data['info']['teams'][0],
        'team2': data['info']['teams'][1],
        'winner': data['info']['outcome'].get('winner', None),
        'result': data['info']['outcome'].get('result', None)
    }
    match_info = {'match_id': create_numerical_id(match_info)} | match_info
    return pd.DataFrame([match_info])

# Initialize an empty DataFrame to store all match data
all_matches_df = pd.DataFrame()

# Iterate over each JSON file in the odis_json_data directory
for filename in os.listdir('odis_json_data'):
    if filename.endswith('.json'):
        filepath = os.path.join('odis_json_data', filename)
        with open(filepath) as file:
            data = json.load(file)
        # Extract match info and append it to the all_matches_df DataFrame
        match_df = extract_match_info(data)
        all_matches_df = pd.concat([all_matches_df, match_df], ignore_index=True)

In [15]:
all_matches_df.head()

,match_id,date,gender,team1,team2,winner,result
0,56308590,2023-12-19,male,India,South Africa,South Africa,None
1,84271760,2010-11-28,male,India,New Zealand,India,None
2,60587676,2009-02-01,male,Australia,New Zealand,New Zealand,None
3,36734120,2021-03-09,female,South Africa,India,India,None
4,70311395,2022-06-11,male,Nepal,United States of America,None,tie


###### Next - Innings Data

The `extract_innings_info` function is crucial for parsing the detailed ball-by-ball innings data from a JSON file and extracting key information necessary for the assessment. This function is particularly important for answering questions related to individual player performance, such as calculating the highest strike rate for batsmen in 2019. The function extracts the following details to answer the corresponding questions:

3. Which players had the highest strike rate as batsmen in 2019? (Note to receive full credit, you need to account for handling extras properly.)
- `Team`: Identifies the team for which the innings were played.
- `Over`: Specifies the over number in the innings.
- `Batter`: The player who faced the delivery.
- `Bowler`: The player who bowled the delivery.
- `Non-striker`: The player at the non-striker's end during the delivery.
- `Runs scored by the batter`: The number of runs scored by the batter from the delivery.
- `Extras`: Additional runs awarded to the batting team due to errors or rule infractions by the bowling team.
- `Total runs`: The total number of runs resulting from the delivery, including runs scored by the batter and any extras.

This function is particularly important for calculating the strike rate of batsmen in 2019, as it provides the necessary data on runs scored and balls faced by each batsman. The strike rate is calculated as the total runs scored by a batsman divided by the number of balls faced, multiplied by 100. To accurately calculate the strike rate, the function needs to account for extras properly, excluding deliveries that resulted in wides or no-balls from the balls faced count.

In [16]:
def extract_innings_info(data, match_id):
    innings_list = []
    for innings in data['innings']:
        team = innings['team']
        for over in innings['overs']:
            over_number = over['over']
            for delivery in over['deliveries']:
                batter = delivery['batter']
                bowler = delivery['bowler']
                non_striker = delivery['non_striker']
                runs_batter = delivery['runs']['batter']
                extras = delivery['runs'].get('extras', 0)
                total_runs = delivery['runs']['total']

                delivery_info = {
                    'match_id': match_id,
                    'team': team,
                    'over': over_number,
                    'batter': batter,
                    'bowler': bowler,
                    'non_striker': non_striker,
                    'runs_batter': runs_batter,
                    'extras': extras,
                    'total_runs': total_runs
                }
                innings_list.append(delivery_info)
    return pd.DataFrame(innings_list)

In [17]:
# Start the timer
start_time = time.time()

# Initialize empty DataFrames to store match and innings data
all_matches_df = pd.DataFrame()
all_innings_df = pd.DataFrame()

# Iterate over each JSON file in the odis_json_data directory
for filename in os.listdir('odis_json_data'):
    if filename.endswith('.json'):
        filepath = os.path.join('odis_json_data', filename)
        with open(filepath) as file:
            data = json.load(file)
        # Extract match info and append it to the all_matches_df DataFrame
        match_df = extract_match_info(data)
        all_matches_df = pd.concat([all_matches_df, match_df], ignore_index=True)
        # Extract innings info using the match_id from match_df and append it to the all_innings_df DataFrame
        match_id = match_df['match_id'].iloc[0]
        innings_df = extract_innings_info(data, match_id)
        all_innings_df = pd.concat([all_innings_df, innings_df], ignore_index=True)

# End the timer
end_time = time.time()

# Calculate the elapsed time
elapsed_time = end_time - start_time

print(f"Execution time: {elapsed_time} seconds")

Execution time: 141.44853901863098 seconds


In [18]:
all_innings_df.head()

,match_id,team,over,batter,bowler,non_striker,runs_batter,extras,total_runs
0,56308590,India,0,RD Gaikwad,N Burger,B Sai Sudharsan,4,0,4
1,56308590,India,0,RD Gaikwad,N Burger,B Sai Sudharsan,0,0,0
2,56308590,India,0,Tilak Varma,N Burger,B Sai Sudharsan,0,0,0
3,56308590,India,0,Tilak Varma,N Burger,B Sai Sudharsan,0,0,0
4,56308590,India,0,Tilak Varma,N Burger,B Sai Sudharsan,0,0,0


In [19]:
all_innings_df.shape

(1533282, 9)

In [20]:
print(all_innings_df["match_id"].nunique())

2889


In [21]:
all_innings_df.head()

,match_id,team,over,batter,bowler,non_striker,runs_batter,extras,total_runs
0,56308590,India,0,RD Gaikwad,N Burger,B Sai Sudharsan,4,0,4
1,56308590,India,0,RD Gaikwad,N Burger,B Sai Sudharsan,0,0,0
2,56308590,India,0,Tilak Varma,N Burger,B Sai Sudharsan,0,0,0
3,56308590,India,0,Tilak Varma,N Burger,B Sai Sudharsan,0,0,0
4,56308590,India,0,Tilak Varma,N Burger,B Sai Sudharsan,0,0,0


###### Some sanity checks before loading the data to SQLite

In [22]:
# Check data types
print("Data types for all_matches_df:")
print(all_matches_df.dtypes)
print("\nData types for all_innings_df:")
print(all_innings_df.dtypes)

Data types for all_matches_df:
match_id     int64
date        object
gender      object
team1       object
team2       object
winner      object
result      object
dtype: object

Data types for all_innings_df:
match_id        int64
team           object
over            int64
batter         object
bowler         object
non_striker    object
runs_batter     int64
extras          int64
total_runs      int64
dtype: object


In [23]:
all_matches_df['date'] = pd.to_datetime(all_matches_df['date'])
print(all_matches_df.dtypes)

match_id             int64
date        datetime64[ns]
gender              object
team1               object
team2               object
winner              object
result              object
dtype: object


In my data preprocessing steps, I carefully considered the data types of each column to ensure compatibility with SQLite and efficient pandas operations. The key changes and considerations were:

1. Date Column: I converted the `date` column in `all_matches_df` from an object type to a datetime type. This change enhances the handling of date-related operations within pandas and provides a more appropriate representation of the data.

2. Categorical Data: Although I identified columns like `gender`, `team1`, `team2`, `winner`, `result`, and `win_by_method` as potential candidates for conversion to categorical data types for memory efficiency, I decided to keep them as object types for simplicity and compatibility with SQLite, which stores categorical data as text.

3. Numeric Data: The columns representing numeric data such as `runs_batter`, `extras`, and `total_runs` were already in integer format, which is suitable for both pandas operations and SQLite storage.

4. Boolean Data: I noted that SQLite does not have a separate boolean data type, so any boolean columns would need to be converted to integers or text. However, my dataset did not contain any boolean columns, so no conversion was necessary in this regard.

Overall, the focus was on ensuring that the data types were appropriate for both pandas operations and SQLite storage, with minimal changes to maintain simplicity and compatibility.

In [24]:
# Check for missing values
print("Missing values in all_matches_df:")
print(all_matches_df.isnull().sum())
print("\nMissing xvalues in all_innings_df:")
print(all_innings_df.isnull().sum())

Missing values in all_matches_df:
match_id       0
date           0
gender         0
team1          0
team2          0
winner       141
result      2748
dtype: int64

Missing xvalues in all_innings_df:
match_id       0
team           0
over           0
batter         0
bowler         0
non_striker    0
runs_batter    0
extras         0
total_runs     0
dtype: int64


In [25]:
# Check for negative values in numeric columns of all_innings_df
negative_values_check = all_innings_df[['runs_batter', 'extras', 'total_runs']].lt(0).any()
print("Negative values in all_innings_df:", negative_values_check)

# Check for invalid dates in all_matches_df
try:
    pd.to_datetime(all_matches_df['date'])
    print("All dates in all_matches_df are valid.")
except ValueError:
    print("Invalid dates found in all_matches_df.")

Negative values in all_innings_df: runs_batter    False
extras         False
total_runs     False
dtype: bool
All dates in all_matches_df are valid.


In the following cell, we create a new SQLite database and connect to it using the sqlite3 library in Python. We then create two tables in the database to store the data from our `all_matches_df` and `all_innings_df` DataFrames:

1. **matches table**: This table stores match-level information such as the match ID, date, gender of the teams, team names, the winner of the match, and the result of the match..

2. **innings table**: This table stores detailed ball-by-ball data for each innings, including the match ID, team name, over number, batter, bowler, non-striker, runs scored by the batter, extras, and total runs for each delivery. The match ID in the innings table is a foreign key that references the match ID in the matches table, ensuring that the innings data is linked to the corresponding match information.

After creating the tables, we commit the changes to the database and close the connection. This setup allows us to store and query the cricket match data efficiently using SQL queries.

In [26]:
# Create a new SQLite database and connect to it
db_name = 'odis_data.db'
conn = sqlite3.connect(db_name)

# Create a cursor object to execute SQL commands
cursor = conn.cursor()

# Creating a table for all_matches_df
cursor.execute('''
    CREATE TABLE IF NOT EXISTS matches (
        match_id INTEGER PRIMARY KEY,
        date TEXT,
        gender TEXT,
        team1 TEXT,
        team2 TEXT,
        winner TEXT,
        result TEXT
    )
''')

# Creating a table for all_innings_df
cursor.execute('''
    CREATE TABLE IF NOT EXISTS innings (
        match_id INTEGER,
        team TEXT,
        over INTEGER,
        batter TEXT,
        bowler TEXT,
        non_striker TEXT,
        runs_batter INTEGER,
        extras INTEGER,
        total_runs INTEGER,
        FOREIGN KEY (match_id) REFERENCES matches(match_id)
    )
''')

# Commit the changes
conn.commit()
# Close the connection when done
conn.close()

In the following cell, we establish a connection to the SQLite database using `sqlalchemy`'s `create_engine` function. We then insert the data from the `all_matches_df` and `all_innings_df` DataFrames into the **matches** and **innings** tables, respectively, in the database. The `if_exists='replace'` argument ensures that existing tables are replaced with the new data, while `index=False` excludes the DataFrame index from being inserted as a column in the tables.

In [27]:
# Create a SQLite database connection
engine = create_engine('sqlite:///odis_data.db')
conn = engine.connect()

# Insert data from all_matches_df into the matches table
all_matches_df.to_sql('matches', conn, if_exists='replace', index=False)

# Insert data from all_innings_df into the innings table
all_innings_df.to_sql('innings', conn, if_exists='replace', index=False)

# Close the connection
conn.close()

In the following cell, we establish a connection to the SQLite database and use SQL queries to retrieve and display data from the **matches** and **innings** tables. The data is stored in DataFrames for easy viewing.

In [28]:
# Create a SQLite database connection
engine = create_engine('sqlite:///odis_data.db')

# Query the matches table and display the result
matches_df = pd.read_sql_query("SELECT * FROM matches", engine)
print("Matches Table:")
display(matches_df)

# Query the innings table and display the result
innings_df = pd.read_sql_query("SELECT * FROM innings", engine)
print("Innings Table:")
display(innings_df)

# Close the connection
engine.dispose()

Matches Table:


,match_id,date,gender,team1,team2,winner,result
0,56308590,2023-12-19 00:00:00.000000,male,India,South Africa,South Africa,None
1,84271760,2010-11-28 00:00:00.000000,male,India,New Zealand,India,None
2,60587676,2009-02-01 00:00:00.000000,male,Australia,New Zealand,New Zealand,None
3,36734120,2021-03-09 00:00:00.000000,female,South Africa,India,India,None
4,70311395,2022-06-11 00:00:00.000000,male,Nepal,United States of America,None,tie
...,...,...,...,...,...,...,...
2884,90185231,2016-07-17 00:00:00.000000,male,Ireland,Afghanistan,Afghanistan,None
2885,16844439,2013-06-08 00:00:00.000000,male,England,Australia,England,None
2886,29733151,2009-10-31 00:00:00.000000,male,India,Australia,India,None
2887,86174835,2019-09-22 00:00:00.000000,male,Namibia,Papua New Guinea,Namibia,None


Innings Table:


,match_id,team,over,batter,bowler,non_striker,runs_batter,extras,total_runs
0,56308590,India,0,RD Gaikwad,N Burger,B Sai Sudharsan,4,0,4
1,56308590,India,0,RD Gaikwad,N Burger,B Sai Sudharsan,0,0,0
2,56308590,India,0,Tilak Varma,N Burger,B Sai Sudharsan,0,0,0
3,56308590,India,0,Tilak Varma,N Burger,B Sai Sudharsan,0,0,0
4,56308590,India,0,Tilak Varma,N Burger,B Sai Sudharsan,0,0,0
...,...,...,...,...,...,...,...,...,...
1533277,51877158,Zimbabwe,40,RW Price,TG Southee,KM Jarvis,0,0,0
1533278,51877158,Zimbabwe,40,RW Price,TG Southee,KM Jarvis,0,0,0
1533279,51877158,Zimbabwe,40,RW Price,TG Southee,KM Jarvis,0,0,0
1533280,51877158,Zimbabwe,40,RW Price,TG Southee,KM Jarvis,0,0,0


The following assert statement checks that all matches with a result of 'no result' have a null value in the `winner` column. This is because matches with 'no result' should not have a winning team.

In [29]:
assert matches_df[matches_df['result'] == 'no result']['winner'].isna().sum() \
        == matches_df[matches_df['result'] == 'no result'].shape[0]

The next assert statement checks that all matches with a result of 'tie' also have a null value in the `winner` column. In the case of a tie, there is no single winning team.

In [30]:
assert matches_df[matches_df['result'] == 'tie']['winner'].isna().sum() \
        == matches_df[matches_df['result'] == 'tie'].shape[0]

The following `execute_query` function connects to a SQLite database, executes a given SQL query, and returns the results as a pandas DataFrame. It ensures the database connection is closed after the query execution.

In [31]:
def execute_query(query):
    # Connect to the database
    conn = sqlite3.connect('odis_data.db')
    
    # Execute the query and store the result in a DataFrame
    df = pd.read_sql_query(query, conn)
    
    # Close the connection
    conn.close()
    
    # Return the result DataFrame
    return df

## Question 2

#### **2a.** The win records (percentage win and total wins) for each team by year and gender, excluding ties, matches with no result, and matches decided by the DLS method in the event that, for whatever reason, the planned innings can’t be completed.

In [32]:
# Get unique values for the 'winnner' column
unique_winners = all_matches_df['winner'].unique()
print("Unique winners:", unique_winners)

Unique winners: ['South Africa' 'India' 'New Zealand' None 'West Indies'
 'United States of America' 'Namibia' 'Ireland' 'Australia' 'Sri Lanka'
 'Pakistan' 'Kenya' 'England' 'Bangladesh' 'Afghanistan' 'Nepal'
 'Netherlands' 'Zimbabwe' 'Scotland' 'United Arab Emirates' 'Thailand'
 'Papua New Guinea' 'Hong Kong' 'Canada' 'Oman' 'Asia XI' 'Jersey'
 'Africa XI' 'Bermuda']


In [33]:
query = """
WITH total_matches AS (
    SELECT
        team,
        gender,
        strftime('%Y', date) AS year,
        COUNT(*) AS total_games
    FROM (
        SELECT team1 AS team, gender, date, winner FROM matches
        UNION ALL
        SELECT team2 AS team, gender, date, winner FROM matches
    ) AS all_teams
    WHERE
        winner IS NOT NULL
    GROUP BY
        team,
        gender,
        year
)
SELECT
    *
FROM
    total_matches tm
ORDER BY total_games DESC
"""

total_matches_df = execute_query(query)
print(total_matches_df.shape)
total_matches_df.head()

(490, 4)


,team,gender,year,total_games
0,Australia,male,2009,36
1,India,male,2023,34
2,England,male,2007,33
3,India,male,2007,33
4,Australia,male,2007,32


In [34]:
def compare_total_games_are_equal_between_dfs(python_df, sql_df):
    # Convert the 'date' column to datetime format if it's not already
    if python_df['date'].dtype != 'datetime64[ns]':
        python_df['date'] = pd.to_datetime(python_df['date'])
    
    # Iterate through each row in the SQL DataFrame
    for index, row in sql_df.iterrows():
        team = row['team']
        gender = row['gender']
        year = row['year']
        total_games_sql = row['total_games']
        
        # Filter the Python DataFrame based on the team, gender, and year
        filtered_df = python_df[((python_df['team1'] == team) | (python_df['team2'] == team)) &
                                (python_df['gender'] == gender) &
                                (python_df['date'].dt.year == int(year)) &
                                (python_df['winner'].notna())]
        
        total_games_python = len(filtered_df)
        
        # Check if the total games match
        if total_games_sql != total_games_python:
            print(f"Mismatch for team: {team}, gender: {gender}, year: {year}")
            print(f"SQL total games: {total_games_sql}, Python total games: {total_games_python}")
            return False
        
    return True

compare_total_games_are_equal_between_dfs(all_matches_df.copy(), total_matches_df)

True

In [35]:
query = """
WITH total_matches AS (
    SELECT
        team,
        gender,
        strftime('%Y', date) AS year,
        COUNT(*) AS total_games
    FROM (
        SELECT team1 AS team, gender, date, winner FROM matches
        UNION ALL
        SELECT team2 AS team, gender, date, winner FROM matches
    ) AS all_teams
    WHERE
        winner IS NOT NULL
    GROUP BY
        team,
        gender,
        year
),
total_wins AS (
    SELECT
        winner AS team,
        gender,
        strftime('%Y', date) AS year,
        COUNT(*) AS total_wins
    FROM
        matches
    WHERE
        winner IS NOT NULL
    GROUP BY
        winner,
        gender,
        year
)
SELECT *
FROM total_wins
"""

total_wins_df = execute_query(query)
print(total_wins_df.shape)
total_wins_df.head()

(420, 4)


,team,gender,year,total_wins
0,Afghanistan,male,2009,1
1,Afghanistan,male,2010,1
2,Afghanistan,male,2014,4
3,Afghanistan,male,2015,7
4,Afghanistan,male,2016,5


In [36]:
def compare_total_wins_are_equal_between_dfs(python_df, sql_df):
    # Convert the 'date' column to datetime format if it's not already
    if python_df['date'].dtype != 'datetime64[ns]':
        python_df['date'] = pd.to_datetime(python_df['date'])
    
    # Iterate through each row in the SQL DataFrame
    for index, row in sql_df.iterrows():
        team = row['team']
        gender = row['gender']
        year = row['year']
        total_wins_sql = row['total_wins']
        
        # Filter the Python DataFrame based on the team, gender, and year, and count the wins
        total_wins_python = python_df[(python_df['winner'] == team) &
                                      (python_df['gender'] == gender) &
                                      (python_df['date'].dt.year == int(year))].shape[0]
        
        # Check if the total wins match
        if total_wins_sql != total_wins_python:
            print(f"Mismatch for team: {team}, gender: {gender}, year: {year}")
            print(f"SQL total wins: {total_wins_sql}, Python total wins: {total_wins_python}")
            return False
        
    return True

compare_total_wins_are_equal_between_dfs(all_matches_df.copy(), total_wins_df)

True

#### Final Query

In [37]:
query = """
WITH total_matches AS (
    SELECT
        team,
        gender,
        strftime('%Y', date) AS year,
        COUNT(*) AS total_games
    FROM (
        SELECT team1 AS team, gender, date, winner FROM matches
        UNION ALL
        SELECT team2 AS team, gender, date, winner FROM matches
    ) AS all_teams
    WHERE
        winner IS NOT NULL
    GROUP BY
        team,
        gender,
        year
),
total_wins AS (
    SELECT
        winner AS team,
        gender,
        strftime('%Y', date) AS year,
        COUNT(*) AS total_wins
    FROM
        matches
    WHERE
        winner IS NOT NULL
    GROUP BY
        winner,
        gender,
        year
)
SELECT
    tm.team,
    tm.gender,
    tm.year,
    COALESCE(tw.total_wins, 0) AS total_wins,
    ROUND((COALESCE(tw.total_wins, 0) * 100.0) / tm.total_games, 2) AS win_percentage
FROM
    total_matches tm
    LEFT JOIN total_wins tw ON tm.team = tw.team AND tm.gender = tw.gender AND tm.year = tw.year
"""

perc_wins_and_total_wins_df = execute_query(query)
print(perc_wins_and_total_wins_df.shape)
perc_wins_and_total_wins_df.head()

(490, 5)


,team,gender,year,total_wins,win_percentage
0,Afghanistan,male,2009,1,100.0
1,Afghanistan,male,2010,1,50.0
2,Afghanistan,male,2012,0,0.0
3,Afghanistan,male,2014,4,40.0
4,Afghanistan,male,2015,7,50.0


##### Explanation
The query above calculates the win records (percentage win and total wins) for each team by year and gender, excluding ties and matches with no result. The query consists of:

1. **total_matches CTE**: Computes total games played by each team, grouped by team, gender, and year. It includes both `team1` and `team2` instances from the **matches** table.

2. **total_wins CTE**: Calculates total wins for each team, also grouped by team, gender, and year, considering only matches with a non-null `winner`.

3. **Final SELECT**: Combines total matches and wins, computing the win percentage as `(total_wins / total_games) * 100`. The `COALESCE` function ensures a zero value for teams with no wins.

**The result is a table with team, gender, year, total wins, and win percentage.**

In [38]:
def compare_total_wins_and_win_percentage_between_dfs(python_df, sql_df):
    # Convert the 'date' column to datetime format if it's not already
    if python_df['date'].dtype != 'datetime64[ns]':
        python_df['date'] = pd.to_datetime(python_df['date'])
    
    # Iterate through each row in the SQL DataFrame
    for index, row in sql_df.iterrows():
        team = row['team']
        gender = row['gender']
        year = row['year']
        total_wins_sql = row['total_wins']
        win_percentage_sql = row['win_percentage']
        
        # Filter the Python DataFrame based on the team, gender, and year
        filtered_df = python_df[((python_df['team1'] == team) | (python_df['team2'] == team)) &
                                (python_df['gender'] == gender) &
                                (python_df['date'].dt.year == int(year)) &
                                (python_df['winner'].notna())]
        
        # Calculate the total wins and win percentage
        total_wins_python = filtered_df[filtered_df['winner'] == team].shape[0]
        total_games_python = filtered_df.shape[0]
        win_percentage_python = round((total_wins_python * 100.0) / total_games_python, 2) if total_games_python > 0 else 0
        
        # Check if the total wins and win percentage match
        if total_wins_sql != total_wins_python or win_percentage_sql != win_percentage_python:
            print(f"Mismatch for team: {team}, gender: {gender}, year: {year}")
            print(f"SQL total wins: {total_wins_sql}, Python total wins: {total_wins_python}")
            print(f"SQL win percentage: {win_percentage_sql}, Python win percentage: {win_percentage_python}")
            return False
        
    return True


compare_total_wins_and_win_percentage_between_dfs(all_matches_df.copy(), perc_wins_and_total_wins_df)

True

The functions `compare_total_games_are_equal_between_dfs`, `compare_total_wins_are_equal_between_dfs`, and `compare_total_wins_and_win_percentage_between_dfs` were created to validate the SQL query results against the original Python DataFrame for accuracy. They ensure that the total games, total wins, and win percentages calculated from the SQL queries match the corresponding values derived from the Python DataFrame for each team, gender, and year, thereby supporting the analysis of win records as specified in the assessment question.

#### **2b.** Which male and female teams had the highest win percentages in 2019?

#### Final Query

In [39]:
query = """
WITH total_matches AS (
    SELECT
        team,
        gender,
        strftime('%Y', date) AS year,
        COUNT(*) AS total_games
    FROM (
        SELECT team1 AS team, gender, date, winner FROM matches
        UNION ALL
        SELECT team2 AS team, gender, date, winner FROM matches
    ) AS all_teams
    WHERE
        winner IS NOT NULL
    GROUP BY
        team,
        gender,
        year
),
total_wins AS (
    SELECT
        winner AS team,
        gender,
        strftime('%Y', date) AS year,
        COUNT(*) AS total_wins
    FROM
        matches
    WHERE
        winner IS NOT NULL
    GROUP BY
        winner,
        gender,
        year
),

win_percentage AS (
    SELECT
        tm.team,
        tm.gender,
        tm.year,
        COALESCE(tw.total_wins, 0) AS total_wins,
        ROUND((COALESCE(tw.total_wins, 0) * 100.0) / tm.total_games, 2) AS win_percentage,
        RANK() OVER (PARTITION BY tm.gender, tm.year ORDER BY COALESCE(tw.total_wins, 0) * 100.0 / tm.total_games DESC) AS rank
    FROM
        total_matches tm
        LEFT JOIN total_wins tw ON tm.team = tw.team AND tm.gender = tw.gender AND tm.year = tw.year
)

SELECT
    gender,
    team,
    year,
    win_percentage
FROM
    win_percentage
WHERE
    year = '2019' AND rank = 1
"""

highest_win_percentage_2019_df = execute_query(query)
print(highest_win_percentage_2019_df.shape)
highest_win_percentage_2019_df.head()

(2, 4)


,gender,team,year,win_percentage
0,female,Australia,2019,100.0
1,male,Netherlands,2019,100.0


##### Explanation
The query above identifies the male and female teams with the highest win percentages in 2019. The query is composed of three parts:

1. **total_matches CTE**: Calculates the total games played by each team in 2019, accounting for both instances where the team is either `team1` or `team2`.

2. **total_wins CTE**: Computes the total wins for each team in 2019. Only matches with a non-null `winner` are considered.

3. **win_percentage CTE** Combines the total matches and wins to calculate the win percentage. It also ranks teams within each gender based on their win percentage using the `RANK()` window function.

The final SELECT statement filters the results to show only the top-ranked team (rank = 1) for each gender in 2019, effectively identifying the male and female teams with the highest win percentages.

According to the query, **Australia's female team and Netherland's male team had the highest win percentage in 2019.**

In [40]:
def compare_highest_win_percentage_2019(python_df, sql_df):
    # Filter for matches in 2019
    python_df['date'] = pd.to_datetime(python_df['date'])
    python_df_2019 = python_df[python_df['date'].dt.year == 2019]

    # Calculate total games for each team
    total_games = python_df_2019.groupby(['team1', 'gender']).size().reset_index(name='total_games')
    total_games = total_games.rename(columns={'team1': 'team'})
    total_games_team2 = python_df_2019.groupby(['team2', 'gender']).size().reset_index(name='total_games')
    total_games_team2 = total_games_team2.rename(columns={'team2': 'team'})
    total_games = pd.concat([total_games, total_games_team2], ignore_index=True)
    total_games = total_games.groupby(['team', 'gender']).sum().reset_index()

    # Calculate total wins for each team
    total_wins = python_df_2019.groupby(['winner', 'gender']).size().reset_index(name='total_wins')
    total_wins = total_wins.rename(columns={'winner': 'team'})

    # Merge total games and total wins
    win_percentage = pd.merge(total_games, total_wins, on=['team', 'gender'], how='left')
    win_percentage['win_percentage'] = (win_percentage['total_wins'] / win_percentage['total_games']) * 100

    # Compare with SQL results
    return win_percentage

result = compare_highest_win_percentage_2019(all_matches_df.copy(), highest_win_percentage_2019_df)
print(result.shape)
result.sort_values(by='win_percentage', ascending=False).head()

(28, 5)


,team,gender,total_games,total_wins,win_percentage
1,Australia,female,11,11.0,100.0
11,Netherlands,male,2,2.0,100.0
6,India,female,12,9.0,75.0
9,Namibia,male,4,3.0,75.0
24,United States of America,male,8,6.0,75.0


The function `compare_highest_win_percentage_2019` was developed to verify the SQL query results against the original Python DataFrame for identifying the male and female teams with the highest win percentages in 2019. Although the SQL query and the function output did not match exactly in format, they both correctly identified Australia (female) and Netherlands (male) as the teams with the highest win percentages of 100% for 2019. The function provided additional context by showing the total games and wins, confirming the accuracy of the SQL query results.

#### **2c.** Which players had the highest strike rate as batsmen in 2019? (Note to receive full credit, you need to account for handling extras properly.)

I modified my `extract_innings_info` function to capture detailed information about different types of extras in cricket data. By extracting individual components of the `extras` field, such as 'wides', 'no_balls', 'legbyes', 'byes', and 'penalty', I added them as separate columns in the DataFrame. This enhancement allows me to distinguish between various types of extras, enabling accurate calculations for cricket statistics, particularly when determining a batsman's strike rate.

In [41]:
def get_unique_extras_types(json_files_directory):
    unique_extras_types = set()
    for filename in os.listdir(json_files_directory):
        if filename.endswith('.json'):
            filepath = os.path.join(json_files_directory, filename)
            with open(filepath) as file:
                data = json.load(file)
                for innings in data['innings']:
                    for over in innings['overs']:
                        for delivery in over['deliveries']:
                            if 'extras' in delivery:
                                for extra_type in delivery['extras'].keys():
                                    unique_extras_types.add(extra_type)
    return unique_extras_types

json_files_directory = 'odis_json_data'
extras_types = get_unique_extras_types(json_files_directory)
print(extras_types)

{'wides', 'penalty', 'legbyes', 'byes', 'noballs'}


In [42]:
def extract_innings_info(data, match_id):
    innings_list = []
    for innings in data['innings']:
        team = innings['team']
        for over in innings['overs']:
            over_number = over['over']
            for delivery in over['deliveries']:
                batter = delivery['batter']
                bowler = delivery['bowler']
                non_striker = delivery['non_striker']
                runs_batter = delivery['runs']['batter']
                total_runs = delivery['runs']['total']

                # Extract individual extras components
                wides = delivery.get('extras', {}).get('wides', 0)
                no_balls = delivery.get('extras', {}).get('noballs', 0)
                legbyes = delivery.get('extras', {}).get('legbyes', 0)
                byes = delivery.get('extras', {}).get('byes', 0)
                penalty = delivery.get('extras', {}).get('penalty', 0)

                # Calculate total extras
                total_extras = wides + no_balls + legbyes + byes + penalty

                delivery_info = {
                    'match_id': match_id,
                    'team': team,
                    'over': over_number,
                    'batter': batter,
                    'bowler': bowler,
                    'non_striker': non_striker,
                    'runs_batter': runs_batter,
                    'wides': wides,
                    'no_balls': no_balls,
                    'legbyes': legbyes,
                    'byes': byes,
                    'penalty': penalty,
                    'total_extras': total_extras,
                    'total_runs': total_runs
                }
                innings_list.append(delivery_info)
    return pd.DataFrame(innings_list)

In [43]:
# Start the timer
start_time = time.time()

# Initialize empty DataFrames to store match and innings data
all_matches_df = pd.DataFrame()
all_innings_df = pd.DataFrame()

# Iterate over each JSON file in the odis_json_data directory
for filename in os.listdir('odis_json_data'):
    if filename.endswith('.json'):
        filepath = os.path.join('odis_json_data', filename)
        with open(filepath) as file:
            data = json.load(file)
        # Extract match info and append it to the all_matches_df DataFrame
        match_df = extract_match_info(data)
        all_matches_df = pd.concat([all_matches_df, match_df], ignore_index=True)
        # Extract innings info using the match_id from match_df and append it to the all_innings_df DataFrame
        match_id = match_df['match_id'].iloc[0]
        innings_df = extract_innings_info(data, match_id)
        all_innings_df = pd.concat([all_innings_df, innings_df], ignore_index=True)

# End the timer
end_time = time.time()

# Calculate the elapsed time
elapsed_time = end_time - start_time

print(f"Execution time: {elapsed_time} seconds")

Execution time: 178.4731879234314 seconds


In [44]:
# Check for any negative values in the columns related to extras
negative_checks = ['wides', 'no_balls', 'legbyes', 'byes', 'penalty']
for col in negative_checks:
    if (all_innings_df[col] < 0).any():
        print(f"Column {col} has negative values.")

In [45]:
# Verify that the sum of individual extras matches the 'total_extras' column
extras_sum = all_innings_df['wides'] + all_innings_df['no_balls'] + all_innings_df['legbyes'] + all_innings_df['byes'] + all_innings_df['penalty']
if not (extras_sum == all_innings_df['total_extras']).all():
    print("Sum of individual extras does not match the 'extras' column.")

In [46]:
# Check for missing values in the columns related to extras
missing_values_check = all_innings_df[negative_checks].isnull().sum()
print(missing_values_check)

wides       0
no_balls    0
legbyes     0
byes        0
penalty     0
dtype: int64


In [47]:
all_innings_df.dtypes

match_id         int64
team            object
over             int64
batter          object
bowler          object
non_striker     object
runs_batter      int64
wides            int64
no_balls         int64
legbyes          int64
byes             int64
penalty          int64
total_extras     int64
total_runs       int64
dtype: object

In the database creation and data insertion process, I updated the **innings** table schema to include columns for each type of extra. This ensures that the detailed extras information extracted by the `extract_innings_info` function is properly stored in the database. The data from `all_matches_df` and `all_innings_df` is then inserted into the corresponding tables in the SQLite database, providing a structured and accessible format for further analysis and querying.

In [48]:
# Create a new SQLite database and connect to it
db_name = 'odis_data.db'
conn = sqlite3.connect(db_name)

# Create a cursor object to execute SQL commands
cursor = conn.cursor()

# Creating a table for all_matches_df
cursor.execute('''
    CREATE TABLE IF NOT EXISTS matches (
        match_id INTEGER PRIMARY KEY,
        date TEXT,
        gender TEXT,
        team1 TEXT,
        team2 TEXT,
        winner TEXT,
        result TEXT,
        win_by_method TEXT
    )
''')

# Creating a table for all_innings_df
cursor.execute('''
    CREATE TABLE IF NOT EXISTS innings (
        match_id INTEGER,
        team TEXT,
        over INTEGER,
        batter TEXT,
        bowler TEXT,
        non_striker TEXT,
        runs_batter INTEGER,
        wides INTEGER,
        no_balls INTEGER,
        legbyes INTEGER,
        byes INTEGER,
        penalty INTEGER,
        total_runs INTEGER,
        FOREIGN KEY (match_id) REFERENCES matches(match_id)
    )
''')

# Commit the changes
conn.commit()
# Close the connection when done
conn.close()

# Create a SQLite database connection
engine = create_engine('sqlite:///odis_data.db')
conn = engine.connect()

# Insert data from all_matches_df into the matches table
all_matches_df.to_sql('matches', conn, if_exists='replace', index=False)

# Insert data from all_innings_df into the innings table
all_innings_df.to_sql('innings', conn, if_exists='replace', index=False)

# Close the connection
conn.close()

In [49]:
# Create a SQLite database connection
engine = create_engine('sqlite:///odis_data.db')

# Query the innings table and display the result
innings_df = pd.read_sql_query("SELECT * FROM innings", engine)
print("Innings Table:")
display(innings_df)

# Close the connection
engine.dispose()

Innings Table:


,match_id,team,over,batter,bowler,non_striker,runs_batter,wides,no_balls,legbyes,byes,penalty,total_extras,total_runs
0,56308590,India,0,RD Gaikwad,N Burger,B Sai Sudharsan,4,0,0,0,0,0,0,4
1,56308590,India,0,RD Gaikwad,N Burger,B Sai Sudharsan,0,0,0,0,0,0,0,0
2,56308590,India,0,Tilak Varma,N Burger,B Sai Sudharsan,0,0,0,0,0,0,0,0
3,56308590,India,0,Tilak Varma,N Burger,B Sai Sudharsan,0,0,0,0,0,0,0,0
4,56308590,India,0,Tilak Varma,N Burger,B Sai Sudharsan,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1533277,51877158,Zimbabwe,40,RW Price,TG Southee,KM Jarvis,0,0,0,0,0,0,0,0
1533278,51877158,Zimbabwe,40,RW Price,TG Southee,KM Jarvis,0,0,0,0,0,0,0,0
1533279,51877158,Zimbabwe,40,RW Price,TG Southee,KM Jarvis,0,0,0,0,0,0,0,0
1533280,51877158,Zimbabwe,40,RW Price,TG Southee,KM Jarvis,0,0,0,0,0,0,0,0


In [50]:
innings_df.head()

,match_id,team,over,batter,bowler,non_striker,runs_batter,wides,no_balls,legbyes,byes,penalty,total_extras,total_runs
0,56308590,India,0,RD Gaikwad,N Burger,B Sai Sudharsan,4,0,0,0,0,0,0,4
1,56308590,India,0,RD Gaikwad,N Burger,B Sai Sudharsan,0,0,0,0,0,0,0,0
2,56308590,India,0,Tilak Varma,N Burger,B Sai Sudharsan,0,0,0,0,0,0,0,0
3,56308590,India,0,Tilak Varma,N Burger,B Sai Sudharsan,0,0,0,0,0,0,0,0
4,56308590,India,0,Tilak Varma,N Burger,B Sai Sudharsan,0,0,0,0,0,0,0,0


#### Final Query

In [51]:
query = """
WITH player_runs AS (
    SELECT
        batter,
        SUM(runs_batter) AS total_runs,
        SUM(CASE WHEN wides = 0 AND no_balls = 0 AND penalty = 0 THEN 1 ELSE 0 END) AS balls_faced
    FROM innings
    JOIN matches ON innings.match_id = matches.match_id
    WHERE strftime('%Y', date) = '2019'
    GROUP BY batter
),
player_strike_rate AS (
    SELECT
        batter,
        total_runs,
        balls_faced,
        (total_runs * 100.0 / balls_faced) AS strike_rate
    FROM player_runs
    WHERE balls_faced > 0
)
SELECT *
FROM player_strike_rate
ORDER BY strike_rate DESC
LIMIT 1;
"""

highest_strike_rate_batsmen = execute_query(query)
print(highest_strike_rate_batsmen.shape)
highest_strike_rate_batsmen.head()

(1, 4)


,batter,total_runs,balls_faced,strike_rate
0,SN Thakur,17,6,283.333333


##### Explanation
The query above calculates the highest strike rate for batsmen in the year 2019 while accounting for legal deliveries (excluding wides, no-balls, and penalties) when counting balls faced. The query consists of the following parts:

1. **player_runs CTE**: This part aggregates the total runs scored by each batter and the total balls faced, excluding deliveries with wides, no-balls, and penalties. These exclusions ensure that only legal deliveries are considered when calculating the strike rate.

2. **player_strike_rate CTE**: This part calculates the strike rate for each batter as `(total runs scored / balls faced) * 100`. It ensures that only batters with at least one legal ball faced are included in the calculation.

3. The final SELECT statement orders the results by strike rate in descending order and limits the output to the top result, effectively identifying the batter with the highest strike rate in 2019.

**SN Thakur had the highest strike rate as batsmen in 2019.**

In [52]:
# Merge the innings dataframe with the matches dataframe to get the date information
innings_with_date = pd.merge(all_innings_df, all_matches_df[['match_id', 'date']], on='match_id')

# Convert 'date' column to datetime format
innings_with_date['date'] = pd.to_datetime(innings_with_date['date'])

# Filter for deliveries in 2019
innings_2019 = innings_with_date[innings_with_date['date'].dt.year == 2019]

# Calculate total runs and balls faced for each batter
player_stats = innings_2019.groupby('batter').agg(
    total_runs=pd.NamedAgg(column='runs_batter', aggfunc='sum'),
    balls_faced=pd.NamedAgg(column='batter', aggfunc=lambda x: sum((innings_2019.loc[x.index, 'wides'] == 0) & (innings_2019.loc[x.index, 'no_balls'] == 0) & (innings_2019.loc[x.index, 'penalty'] == 0)))
)

# Calculate strike rate
player_stats['strike_rate'] = player_stats['total_runs'] * 100 / player_stats['balls_faced']

# Get the player with the highest strike rate
highest_strike_rate_player = player_stats.sort_values(by='strike_rate', ascending=False).head(1)
print(highest_strike_rate_player)

           total_runs  balls_faced  strike_rate
batter                                         
SN Thakur          17            6   283.333333


I conducted a sanity check on the SQL query results for calculating the highest strike rate in 2019 by using a Python approach. I merged the `innings` and `matches` dataframes to incorporate the date information into the innings data. I then filtered for deliveries in the year 2019 and grouped the data by batter to aggregate the total runs scored and the number of legal balls faced (excluding wides, no-balls, and penalties). Using this aggregated data, I calculated the strike rate for each batter and identified the batter with the highest strike rate. This Python-based approach served as a validation for the correctness of the SQL query results, ensuring the accuracy of the analysis.

## Question 3
#### Please provide a brief written answer to the following question. The coding assessment focused on a batch backfilling use case. If the use case was extended to required incrementally loading new match data on a go-forward basis, how would your solution change?

In the coding assessment, the focus was on a batch backfilling use case, where the entire dataset is processed and loaded into the database at once. This approach is suitable for situations where the historical data needs to be loaded initially. However, if the use case was extended to require incrementally loading new match data on a go-forward basis, my solution would need to adapt to handle new match data as it becomes available, rather than processing all the data in a batch. Here's how I would modify my solution:

- Data Ingestion: Instead of downloading and processing all the files at once, I would implement a mechanism to check for new or updated files on the source (cricsheet.org). This could be done by tracking the last processed file or date and only downloading files that are newer. If available, I could also use APIs provided by cricsheet.org to receive notifications of new or updated data.

- Database Schema: The database schema might need to be adjusted to include a field for the data ingestion timestamp or a flag to indicate whether a record has been processed. This would help in identifying new or updated records that need to be processed.

- Data Processing: The data processing step would need to be modified to handle incremental data. Instead of truncating and reloading the entire dataset, I would implement a process to insert or update only the new or changed records in the database. This might involve checking for existing records based on a unique identifier (e.g., match_id) and updating them if necessary, or inserting new records.

- Automation: I would set up a scheduled task or cron job to run the data ingestion and processing script at regular intervals (e.g., daily or weekly) to ensure that the database is kept up-to-date with the latest match data. For more sophisticated automation, I could consider using cloud services or workflow orchestration tools like Apache Airflow.

- Error Handling and Logging: For an incremental loading process, robust error handling and logging mechanisms are essential to capture any issues that occur during data ingestion or processing. This helps in troubleshooting and ensuring data integrity. Additionally, implementing data quality checks is crucial to validate the accuracy and completeness of the newly ingested data. These checks might include verifying the format and range of data values, checking for duplicates, and ensuring that foreign key relationships are maintained. By incorporating these data quality checks, we can further enhance the reliability and consistency of the incremental loading process.

By implementing these changes, my solution would be able to handle incremental loading of new match data, keeping the database up-to-date with minimal manual intervention.